In [16]:
# Pandas, matplotlib, random
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import pickle

# Sklearn
from sklearn import tree
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer

from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


# Generic

In [48]:
DEFAULT_K = 10 # default number of folds

def holisticsTrainTest(dataframe, list_complete_participants, train_test_split = 0.8):
    """
    Prepare a dataframe and split it into 70% training and 30% test. Return both sets.
    It's assumed the dataframe does have the participant_no feature
    """
    
    df = dataframe.copy()

    random.seed(75)
    random.shuffle(list_complete_participants)
    random.seed(75)
    test_participants = random.sample(set(list_complete_participants), 
                                      int(round((1 - train_test_split) * len(list_complete_participants))))
#     print(test_participants)
    print(len(test_participants))

    # only pick the 30% of the complete participants for testing
    df_test = df[df['Participant_No'].isin(test_participants)]

    print("Testing on participants:")
    print(df_test['Participant_No'].unique())
    
    # use the rest for training (the negate of above)
    df_train = df[~df['Participant_No'].isin(test_participants)]
        
    # removing the participant number since it's a holistic model
#     del df['Participant_No']
    del df_test['Participant_No']
    del df_train['Participant_No']
         
    # shuffle 
    df_train = df_train.sample(frac=1, random_state=100).reset_index(drop=True)
    df_test = df_test.sample(frac=1, random_state=100).reset_index(drop=True)
    
#     # determine split
#     idx_split = int(df.shape[0] * train_test_split)
    
#     # split the dataframe
#     df_train = df.iloc[:idx_split, :]
#     df_test = df.iloc[idx_split:, :]
    
    # create binary label versions of the sets
    df_train_binary = df_train.copy()
    df_test_binary = df_test.copy()
    df_train_binary['Discrete Thermal Comfort_TA'] = df_train['Discrete Thermal Comfort_TA'].map(lambda x: 1 if x != 0 else 0)
    df_test_binary['Discrete Thermal Comfort_TA'] = df_test['Discrete Thermal Comfort_TA'].map(lambda x: 1 if x != 0 else 0)

    return df_train, df_test, df_train_binary, df_test_binary

def LOPOTrainTest(dataframe, list_participants_65, train_test_split = 0.8, with_labels=False):
    """
    Prepare the dataframes and split into 80% of participants for training and 20% for testing. However, technically
    there are a bit more than 65 participants in the dataframe but we are only using 65 as the number for train-test
    split. Thus, the remaining participants, with fewer responses, will be used for training purposes, and will also
    be ignored for cv later on
    """

    test_participants = random.sample(set(list_participants_65), int(round((1- train_test_split) * len(list_participants_65))))

    # only pick the 20% of the 65 participants for testing
    df_test = dataframe[dataframe['Participant_No'].isin(test_participants)]
    # use the rest for training (the negate of above)
    df_train = dataframe[~dataframe['Participant_No'].isin(test_participants)]

    if with_labels:
        X_train = np.array(df_train.iloc[:, 0:df_train.shape[1] - 1]) # minus 1 for the comfort label
        y_train = np.array(df_train['Discrete Thermal Comfort_TA'])

        X_test = np.array(df_test.iloc[:, 0:df_test.shape[1] - 1]) # minus 1 for the comfort label
        y_test = np.array(df_test['Discrete Thermal Comfort_TA'])
        
        return X_train, X_test, y_train, y_test
    else:
        return df_train, df_test

def LOPO_cv(X, list_participants_65):
    """
    Workst as a cross-validation generator. It returns an iterable of length n_folds, each element of which is a 
    2-tuple of numpy 1-d arrays (train_index, test_index) containing the indices of the test and training sets for 
    that cross-validation run. 
    The number of folds will be equal to the number of participants in dataframe
    """
    
    # make sure the splits are only based on participants with complete data
    existing_p_in_X = list(set(X[:, 0]).intersection(list_participants_65))
    n_folds = len(existing_p_in_X)
    
    # CV array of 2-tuples
    lopo_CViterator = []
    
    # auxiliary dataframe just so we can get the indices later
    aux_df = pd.DataFrame(X)
    
    # TODO: DEBUGGING
#     print(existing_p_in_X)
#     print(aux_df)
    
    for i in range(n_folds):
        # use current participant to separate as the test set
        curr_participant = existing_p_in_X[i]
        test_indices = aux_df[aux_df.iloc[:,0] == curr_participant].index.values.astype(int)
        # the rest participants will be for train
        train_indices = aux_df[aux_df.iloc[:,0] != curr_participant].index.values.astype(int)

        #TODO: DEBUGGING
#         print(test_idx)
#         print(train_idx)

        # append both indices to the iterable
        lopo_CViterator.append( (train_indices, test_indices) )
    
    return lopo_CViterator

def predictions_to_temp(predictions, df_train, df_test):
    temp_range_train = [ df_train['Temperature (Fahrenheit)'].min(), df_train['Temperature (Fahrenheit)'].max()]
    temp_range_test = [ df_test['Temperature (Fahrenheit)'].min(), df_test['Temperature (Fahrenheit)'].max()]
    return


def chooseK(train_labels):
    """
    Determine number of folds
    """
    DEFAULT_K = 10
    
    classCounter = Counter(train_labels)
    numLeastCommonClass = min(classCounter.values())
    return min(numLeastCommonClass, DEFAULT_K)

def getClfMetrics(test_labels, pred_labels):
    """
    Compute different validation metrics for a classification problem.
    Metrics:
    - micro and macro F1 score
    - Confusion Matrix
    - Classification Report
    """
    
    acc = accuracy_score(test_labels, pred_labels) 
    print("\nAccuracy (f1 micro) on test set: ", acc)
    print("F1 micro on test set: ", f1_score(test_labels, pred_labels, average = 'micro'))
    print("F1 macro on test set: ", f1_score(test_labels, pred_labels, average = 'macro'))
    print("\nConfusion Matrix: ")
    print(confusion_matrix(test_labels, pred_labels)) #, labels = [-2, -1, 0, 1, 2])
    print("\nClassification Metrics: ")
    print(classification_report(test_labels, pred_labels))
    return acc

# Baselines

In [49]:
def calculateCost(dataframe, list_complete_participants, temp_high, temp_low, clf=None, occuTherm=False):
    
    train_test_split = 0.8

    random.seed(75)
    random.shuffle(list_complete_participants)
    random.seed(75)
    test_participants = random.sample(set(list_complete_participants), 
                                      int(round((1 - train_test_split) * len(list_complete_participants))))
    
    
    # take the same 30% used for data split
    df_test = dataframe[dataframe['Participant_No'].isin(test_participants)]

    # the temperature range we care about is such that all the baselines predicts comfortable
    pred_comfort = 0
    
    # get list of participants in the test set
    participants = list(df_test['Participant_No'].unique())

    print("Testing on participants:")
    print(df_test['Participant_No'].unique())
        
    accumulative_rmse = 0 # accumulative RMSE
    total_rmse = 0
    total_num_responses = 0
    
    # for each participant
    for p in participants:
        curr_participant = df_test[df_test['Participant_No'] == p]
        
        # RMSE calculation variables
        participant_rmse = 0 
        participant_num_respones = 0

        # if we are evaluating our model, we need to find the comfort temp range for each participant
        if occuTherm:
            curr_p_occutherm = curr_participant.copy()
            
            # for prediction we don't use these columns
            del curr_p_occutherm['Discrete Thermal Comfort_TA']
            del curr_p_occutherm['Participant_No']
            
            # calculate the range of temperature at which occuTherm model predicts 0
            
            occutherm_pred = clf.predict(curr_p_occutherm)
                        
            occutherm_temps = []
            curr_p_occutherm = curr_p_occutherm.reset_index(drop=True)
            
            for index, row in curr_p_occutherm.iterrows():
                temp = row['Temperature (Fahrenheit)']
                if occutherm_pred[index] == 0:
                    occutherm_temps.append(temp)
        
            temp_low = min(occutherm_temps)  # fahrenheit
            temp_high = max(occutherm_temps) # fahrenheit

            print(temp_low)
            print(temp_high)
            
        # iterate through all the participant's responses
        for index, row in curr_participant.iterrows():
            temp = row['Temperature (Fahrenheit)']
            true_comfort = row['Discrete Thermal Comfort_TA']
            
            # see if temperature falls within range of the baseline
            if (temp >= temp_low) and (temp <= temp_high):
                # keep track of current participant's cost
                participant_rmse = participant_rmse + (pred_comfort - true_comfort) ** 2
                print((pred_comfort - true_comfort)**2)
                participant_num_respones = participant_num_respones + 1
                
        # end of current participant's responses
        
        # continue to add with the following participants
        if participant_num_respones != 0:
            total_rmse = total_rmse + participant_rmse # add
            total_num_responses = total_num_responses + participant_num_respones
            # rmse for current participant
            participant_rmse = math.sqrt(participant_rmse / participant_num_respones)

        # add participants RMSE
        accumulative_rmse = accumulative_rmse + participant_rmse

        # calculate rmse of current participant
        print("Num Participants responses: {}".format(curr_participant.shape[0]))


    # all participants processed
    total_rmse = math.sqrt(total_rmse / total_num_responses)
    
    print("Total RMSE across all participants: {}".format(total_rmse))
    print("Accumulative of RMSE of each participant: {}".format(accumulative_rmse))
    
    return total_rmse, accumulative_rmse


# Models

In [4]:
# From https://github.com/CenterForTheBuiltEnvironment/comfort_tool/blob/master/contrib/comfort_models.py
def comfPMV(ta, tr, vel, rh, met, clo, wme):
    """
    returns [pmv, ppd]
    ta, air temperature (C)
    tr, mean radiant temperature (C)
    vel, relative air velocity (m/s)
    rh, relative humidity (%) Used only this way to input humidity level
    met, metabolic rate (met)
    clo, clothing (clo)
    wme, external work, normally around 0 (met)
    """

    pa = rh * 10 * math.exp(16.6536 - 4030.183 / (ta + 235))

    icl = 0.155 * clo  # thermal insulation of the clothing in M2K/W
    m = met * 58.15  # metabolic rate in W/M2
    w = wme * 58.15  # external work in W/M2
    mw = m - w  # internal heat production in the human body
    if (icl <= 0.078):
        fcl = 1 + (1.29 * icl)
    else:
        fcl = 1.05 + (0.645 * icl)

    # heat transf. coeff. by forced convection
    hcf = 12.1 * math.sqrt(vel)
    taa = ta + 273
    tra = tr + 273
    tcla = taa + (35.5 - ta) / (3.5 * icl + 0.1)

    p1 = icl * fcl
    p2 = p1 * 3.96
    p3 = p1 * 100
    p4 = p1 * taa
    p5 = (308.7 - 0.028 * mw) + (p2 * math.pow(tra / 100, 4))
    xn = tcla / 100
    xf = tcla / 50
    eps = 0.00015

    n = 0
    while abs(xn - xf) > eps:
        xf = (xf + xn) / 2
        hcn = 2.38 * math.pow(abs(100.0 * xf - taa), 0.25)
        if (hcf > hcn):
            hc = hcf
        else:
            hc = hcn
        xn = (p5 + p4 * hc - p2 * math.pow(xf, 4)) / (100 + p3 * hc)
        n += 1
        if (n > 150):
            print('Max iterations exceeded')
            return 1


    tcl = 100 * xn - 273

    # heat loss diff. through skin
    hl1 = 3.05 * 0.001 * (5733 - (6.99 * mw) - pa)
    # heat loss by sweating
    if mw > 58.15:
        hl2 = 0.42 * (mw - 58.15)
    else:
        hl2 = 0
    # latent respiration heat loss
    hl3 = 1.7 * 0.00001 * m * (5867 - pa)
    # dry respiration heat loss
    hl4 = 0.0014 * m * (34 - ta)
    # heat loss by radiation
    hl5 = 3.96 * fcl * (math.pow(xn, 4) - math.pow(tra / 100, 4))
    # heat loss by convection
    hl6 = fcl * hc * (tcl - ta)

    ts = 0.303 * math.exp(-0.036 * m) + 0.028
    pmv = ts * (mw - hl1 - hl2 - hl3 - hl4 - hl5 - hl6)
    ppd = 100.0 - 95.0 * math.exp(-0.03353 * pow(pmv, 4.0)
        - 0.2179 * pow(pmv, 2.0))

    r = []
    r.append(pmv)
    r.append(ppd)

    return r

In [5]:
def ppv(pmv, dataframe_train, dataframe_test, ta, tr, vel, rh, met, clo, binary=False, no_clo=False):
    """
    returns ppv as a linear combination of the following features:
    ta, air temperature (C)
    tr, mean radiant temperature (C)
    vel, relative air velocity (m/s)
    rh, relative humidity (%) Used only this way to input humidity level
    met, metabolic rate (met)
    clo, clothing (clo)
    
    According to SPOT paper:
    ppv(x) = pmv(x) + personal(x)

    personal(x) = a'.x + b

    a={a_temp, a_radiant, a_velocity, a_humidity, a_metabolic, a_clothing}
    """
    
    ppv_train_df = pd.DataFrame()
    ppv_test_df = pd.DataFrame()
    
    ppv_train_df['Temperature (Fahrenheit)'] = dataframe_train['Temperature (Fahrenheit)']
    ppv_train_df['tr'] = 25
    ppv_train_df['vel'] = 0.2
    ppv_train_df['rh'] = 0.6
    ppv_train_df['met'] = 1.1
    ppv_train_df['Discrete Thermal Comfort_TA'] = dataframe_train['Discrete Thermal Comfort_TA']

    ppv_test_df['Temperature (Fahrenheit)'] = dataframe_test['Temperature (Fahrenheit)']
    ppv_test_df['tr'] = 25
    ppv_test_df['vel'] = 0.2
    ppv_test_df['rh'] = 0.6
    ppv_test_df['met'] = 1.1
    ppv_test_df['Discrete Thermal Comfort_TA'] = dataframe_test['Discrete Thermal Comfort_TA']
        
    if no_clo:
        ppv_train_df['ClothingInsulation'] = 0.8
        ppv_test_df['ClothingInsulation'] = 0.8
    else:
        ppv_train_df['ClothingInsulation'] = dataframe_train['ClothingInsulation']
        ppv_test_df['ClothingInsulation'] = dataframe_test['ClothingInsulation']
    
    # remapping for binary 
    if binary:
        ppv_train_df['Discrete Thermal Comfort_TA'] = ppv_train_df['Discrete Thermal Comfort_TA'].map(lambda x: 1 if x != 0 else 0)
        ppv_test_df['Discrete Thermal Comfort_TA'] = ppv_test_df['Discrete Thermal Comfort_TA'].map(lambda x: 1 if x != 0 else 0)
        
    # break down the df
    X_train = np.array(ppv_train_df.iloc[:, 0:ppv_train_df.shape[1] - 1]) # minus 1 for the comfort label
    y_train = np.array(ppv_train_df['Discrete Thermal Comfort_TA'])
    
    X_test = np.array(ppv_test_df.iloc[:, 0:ppv_test_df.shape[1] - 1]) # minus 1 for the comfort label
    y_test = np.array(ppv_test_df['Discrete Thermal Comfort_TA'])
        
    # train linear regression
    reg = LinearRegression().fit(X_train, y_train)

    personal_pred = reg.predict(X_test)
    
    ppv_pred = np.around(pmv + personal_pred, 0)
    
    # clip to -2,+2
    ppv_pred = np.clip(ppv_pred, a_min = -2, a_max = 2) 
    
    # clip to 0,1
    if binary:
        ppv_pred = np.clip(ppv_pred, a_min = 0, a_max = 1) 
    
    # get metrics
    return getClfMetrics(y_test, ppv_pred)


In [6]:
def ppv_value(pmv, dataframe_train, dataframe_test, ta, tr, vel, rh, met, clo, binary=False, no_clo=False):
    """
    returns ppv as a linear combination of the following features:
    ta, air temperature (C)
    tr, mean radiant temperature (C)
    vel, relative air velocity (m/s)
    rh, relative humidity (%) Used only this way to input humidity level
    met, metabolic rate (met)
    clo, clothing (clo)
    
    According to SPOT paper:
    ppv(x) = pmv(x) + personal(x)

    personal(x) = a'.x + b

    a={a_temp, a_radiant, a_velocity, a_humidity, a_metabolic, a_clothing}
    """
    
    ppv_train_df = pd.DataFrame()
    ppv_test_df = pd.DataFrame()
    
    ppv_train_df['Temperature (Fahrenheit)'] = dataframe_train['Temperature (Fahrenheit)']
    ppv_train_df['tr'] = 25
    ppv_train_df['vel'] = 0.2
    ppv_train_df['rh'] = 0.6
    ppv_train_df['met'] = 1.1
    ppv_train_df['Discrete Thermal Comfort_TA'] = dataframe_train['Discrete Thermal Comfort_TA']

    ppv_test_df['Temperature (Fahrenheit)'] = dataframe_test['Temperature (Fahrenheit)']
    ppv_test_df['tr'] = 25
    ppv_test_df['vel'] = 0.2
    ppv_test_df['rh'] = 0.6
    ppv_test_df['met'] = 1.1
    ppv_test_df['Discrete Thermal Comfort_TA'] = dataframe_test['Discrete Thermal Comfort_TA']
        
    if no_clo:
        ppv_train_df['ClothingInsulation'] = 0.8
        ppv_test_df['ClothingInsulation'] = 0.8
    else:
        ppv_train_df['ClothingInsulation'] = dataframe_train['ClothingInsulation']
        ppv_test_df['ClothingInsulation'] = dataframe_test['ClothingInsulation']
    
    # remapping for binary 
    if binary:
        ppv_train_df['Discrete Thermal Comfort_TA'] = ppv_train_df['Discrete Thermal Comfort_TA'].map(lambda x: 1 if x != 0 else 0)
        ppv_test_df['Discrete Thermal Comfort_TA'] = ppv_test_df['Discrete Thermal Comfort_TA'].map(lambda x: 1 if x != 0 else 0)
        
    # break down the df
    X_train = np.array(ppv_train_df.iloc[:, 0:ppv_train_df.shape[1] - 1]) # minus 1 for the comfort label
    y_train = np.array(ppv_train_df['Discrete Thermal Comfort_TA'])
    
    X_test = np.array(ppv_test_df.iloc[:, 0:ppv_test_df.shape[1] - 1]) # minus 1 for the comfort label
    y_test = np.array(ppv_test_df['Discrete Thermal Comfort_TA'])
        
    # train linear regression
    reg = LinearRegression().fit(X_train, y_train)

    personal_pred = reg.predict(X_test) # we want the ppv value or the current training
    
    ppv_pred = pmv + personal_pred
    
    # clip to -2,+2
    ppv_pred = np.clip(ppv_pred, a_min = -2, a_max = 2) 
    
    # clip to 0,1
    if binary:
        ppv_pred = np.clip(ppv_pred, a_min = 0, a_max = 1) 
    
    return ppv_pred


In [7]:
def selectModelParameters(train_vectors, train_labels, trainclf, parameters, scorer, useSampleWeight = False, 
                          list_participants_65 = None, LOPO=False):
    """
    Choose the best combination of parameters for a given model
    """
    
    k = chooseK(train_labels) # get number of folds

    stratifiedKFold = StratifiedKFold(n_splits = k)
    if useSampleWeight:
        n_samples = len(train_labels)
        n_classes = len(set(train_labels))
        classCounter = Counter(train_labels)
        sampleWeights = [n_samples / (n_classes * classCounter[label]) for label in train_labels]
        
        if LOPO:
            chosen_cv = LOPO_cv(train_vectors, list_participants_65)
            # train_vectors still have the Participant_No, so remove the first column
            train_vectors = train_vectors[:,1:]
            print("Number of folds: " + str(len(chosen_cv)))
        else:
            print("Number of folds: " + str(k))
            chosen_cv = stratifiedKFold
        
        gridSearch = GridSearchCV(trainclf, parameters, cv = chosen_cv, scoring = scorer, 
                                      fit_params = {'sample_weight' : sampleWeights})
    else:
        if LOPO:
            chosen_cv = LOPO_cv(train_vectors, list_participants_65)
            # train_vectors still have the Participant_No, so remove the first column
            train_vectors = train_vectors[:,1:]
            print("Number of folds: " + str(len(chosen_cv)))
        else:
            print("Number of folds: " + str(k))
            chosen_cv = stratifiedKFold
        
        gridSearch = GridSearchCV(trainclf, parameters, cv = chosen_cv, scoring = scorer)
    
    gridSearch.fit(train_vectors, train_labels)
    print("Best parameters set found on development set:")
    print(gridSearch.best_params_)
    
    return gridSearch.best_estimator_

In [8]:
def trainTest_tunedModel(df_train, df_test, clf_optimal, LOPO=False, mlp_tuned=False, binary_mlp=False):
    # transform unseen test set and whole train split
    X_train = np.array(df_train.iloc[:, 0:df_train.shape[1] - 1]) # minus 1 for the comfort label
    y_train = np.array(df_train['Discrete Thermal Comfort_TA'])

    X_test = np.array(df_test.iloc[:, 0:df_test.shape[1] - 1]) # minus 1 for the comfort label
    y_test = np.array(df_test['Discrete Thermal Comfort_TA'])

    # remapping for mlp
    if mlp_tuned:
        # re-map labels from {-2, -1, 0, 1, 2} to {0, 1, 2, 3, 4}; works better with _tanh_ activation
        y_norm_train = y_train + 2
        y_norm_test = y_test + 2
    
        # binary case
        if binary_mlp:
            y_norm_train = df_train['Discrete Thermal Comfort_TA'].map(lambda x: 1 if x != 0 else 0)
            y_norm_test = df_test['Discrete Thermal Comfort_TA'].map(lambda x: 1 if x != 0 else 0)
    
        # generate one-hot vector representation; e.g., 0 = [1 0 0 0 0], 1 = [0 1 0 0 0], etc.
        y_train = np.zeros((y_norm_train.size, y_norm_train.max()+1)) 
        y_train[np.arange(y_norm_train.size),y_norm_train] = 1 

        y_test = np.zeros((y_norm_test.size, y_norm_test.max()+1)) 
        y_test[np.arange(y_norm_test.size),y_norm_test] = 1 

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train) # TODO: does this take care of boolean variables?
        X_test = scaler.fit_transform(X_test)
        
    if LOPO:
        # X datasets still have the Participant_No, so remove the first column
        X_train = X_train[:,1:]
        X_test = X_test[:,1:]
        
    # retrain in all train split with the tuned model
    clf_optimal.fit(X_train, y_train)

    #predict the response on test set
    y_pred = clf_optimal.predict(X_test)

    if mlp_tuned:
        # reverse one hot encoding
        y_pred_normal = []
        y_test_normal = []

        if binary_mlp:
            for i in range(0,len(y_test)):
                if y_pred[i,0] == 1:
                    y_pred_normal.append(0)
                elif y_pred[i,1] == 1:
                    y_pred_normal.append(1)
                else:
                    y_pred_normal.append(0)
                    
                if y_test[i,0] == 1:
                    y_test_normal.append(0)
                elif y_test[i,1] ==1:
                    y_test_normal.append(1)
                else:
                    y_pred_normal.append(0)

        else: 
            for i in range(0,len(y_test)):
                if y_pred[i,0] == 1:
                    y_pred_normal.append(-2)
                elif y_pred[i,1] == 1:
                    y_pred_normal.append(-1)
                elif y_pred[i,2] == 1:
                    y_pred_normal.append(0)
                elif y_pred[i,3] == 1:
                    y_pred_normal.append(1)
                elif y_pred[i,4] == 1:
                    y_pred_normal.append(2)
                else:
                    y_pred_normal.append(0)

                if y_test[i,0] == 1:
                    y_test_normal.append(-2)
                elif y_test[i,1] ==1:
                    y_test_normal.append(-1)
                elif y_test[i,2] == 1:
                    y_test_normal.append(0)
                elif y_test[i,3] == 1:
                    y_test_normal.append(1)
                elif y_test[i,4] == 1:
                    y_test_normal.append(2)
                else:
                    y_pred_normal.append(0)
    

        y_pred= np.array(y_pred_normal)
        y_test = np.array(y_test_normal)

    
    # get metrics
    return getClfMetrics(y_test, y_pred), y_pred


In [9]:
def buildTrainRF(dataframe, list_participants_65 = None, LOPO=False):
    """
    Breakdown the dataframe into X and y arrays. Later split them in train and test set. Train the model with CV
    and then find the optimal tree depth and report the accuracy
    """
    
    # create design matrix X and target vector y
    X = np.array(dataframe.iloc[:, 0:dataframe.shape[1] - 1]) # minus 1 for the comfort label
    y = np.array(dataframe['Discrete Thermal Comfort_TA'])

    print("Features: {}".format(dataframe.columns.values[:-1]))  # minus 1 for the comfort label

    parameters = {'n_estimators' : [10, 100, 1000],
                  'criterion' : ['entropy', 'gini'],
                  'min_samples_split' : [2, 10, 20, 30], 
                  'class_weight' : ['balanced', 'balanced_subsample']}
    scorer = 'f1_micro'
    clf = RandomForestClassifier(n_estimators = 10, min_samples_split = 2, class_weight = 'balanced', 
                                 random_state = 100)
    
    # --------------------------------------------
    # Leave-one-person-out cross-validation (LOPO)
    # --------------------------------------------
    if LOPO:
        # split into train and test (they still contain the Participant_No as a column)
        X_train, X_test, y_train, y_test = LOPOTrainTest(dataframe, list_participants_65, with_labels=True)
        rf_classifier = selectModelParameters(X_train, y_train, clf, parameters, scorer, 
                                              list_participants_65=list_participants_65, LOPO=True)

        # find optimal depth and generate model
        optimal_depth = chooseOptimalTreeDepth(rf_classifier, X_train, y_train, 
                                               list_participants_65=list_participants_65, LOPO=True)
        
        # X datasets still have the Participant_No, so remove the first column
        X_train = X_train[:,1:]
        X_test = X_test[:,1:]

    else:
         # split into train and test CV
        test_size_percentage = 0.2

        # X_train = train + cv set (train_vectors)
        # X_test = test set (test_vectors)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size_percentage, 
                                                        random_state = 100, stratify = y)
        # for the holistic approach, dataframe doesn't have the Participants_No column so proceed to do gridsearchCV
        rf_classifier = selectModelParameters(X_train, y_train, clf, parameters, scorer)
        
        # find optimal depth and generate model
        optimal_depth = chooseOptimalTreeDepth(rf_classifier, X_train, y_train)

    # generate the model with the selected paramters plus the optimal depth and do the model fitting
    rf_optimal = rf_classifier.set_params(max_depth = optimal_depth)
    print(rf_optimal)
    
    # fit the model
    rf_optimal.fit(X_train, y_train)

    # predict the response on test set
    y_pred = rf_optimal.predict(X_test)

    # get metrics
    rf_acc = getClfMetrics(y_test, y_pred)
    
    return rf_acc, rf_optimal

# Choose the optimal depth of a tree model 
def chooseOptimalTreeDepth(clf, train_vectors, train_labels, list_participants_65 = None,
                           LOPO=False, plot = True):
    """
    Choose the optimal depth of a tree model 
    """
    
    DEFAULT_K = 10
    
    # generate a list of potential depths to calculate the optimal
    depths = list(range(1, 25))

    # empty list that will hold cv scores
    cv_scores = []

    print("Finding optimal tree depth")
    # find optimal tree depth    
    for d in depths: # TODO: try using chooseK(train_labels) instead of jus DEFAULT_K
        clf_depth = clf.set_params(max_depth = d) # use previous parameters while changing depth

        if LOPO:
            chosen_cv = LOPO_cv(train_vectors, list_participants_65)
        else:
            chosen_cv = chooseK(train_labels)

        scores = cross_val_score(clf_depth, train_vectors, 
                                 train_labels, cv = chosen_cv,
                                 scoring = 'accuracy') # accuracy here is f1 micro
        cv_scores.append(scores.mean())

    # changing to misclassification error and determining best depth
    MSE = [1 - x for x in cv_scores] # MSE = 1 - f1_micro
    optimal_depth = depths[MSE.index(min(MSE))]
    print("The optimal depth is: ", optimal_depth, "\n")
    print("Expected accuracy (f1 micro) based on Cross-Validation: ", cv_scores[depths.index(optimal_depth)], "\n")
    
    if plot:
        # plot misclassification error vs depths
        fig = plt.figure(figsize=(12, 10))
        plt.plot(depths, MSE)
        plt.xlabel('Tree Depth', fontsize = 20)
        plt.ylabel('Misclassification Error', fontsize = 20)
        plt.legend(fontsize = 15)
        plt.show()

    return optimal_depth


In [10]:
def buildTrainKNN(dataframe, list_participants_65=None, LOPO=False):
    """
    Breakdown the dataframe into X and y arrays. Later split them in train and test set. Train the model with CV
    and report the accuracy
    """
    
    # create design matrix X and target vector y
    X = np.array(dataframe.iloc[:, 0:dataframe.shape[1] - 1]) # minus 1 for the comfort label
    y = np.array(dataframe['Discrete Thermal Comfort_TA'])

    print("Features: {}".format(dataframe.columns.values[:-1]))  # minus 1 for the comfort label
    
    # split into train and test
    test_size_percentage = 0.2 

    # X_train = train + cv set (train_vectors)
    # X_test = test set (test_vectors)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size_percentage, random_state = 100, stratify = y)

    parameters = {'n_neighbors' : [3, 5, 7, 9, 10, 11, 12, 13, 14, 15], 
                  'weights' : ['uniform', 'distance'], 
                  'metric' : ['seuclidean'], 'algorithm' : ['brute']}
    scorer = 'f1_micro'
    clf = KNeighborsClassifier(n_neighbors = 3, weights = 'uniform', metric = 'seuclidean', algorithm = 'brute')
    
    # --------------------------------------------
    # Leave-one-person-out cross-validation (LOPO)
    # --------------------------------------------
    if LOPO:
        # split into train and test (they still contain the Participant_No as a column)
        X_train, X_test, y_train, y_test = LOPOTrainTest(dataframe, list_participants_65, with_labels=True)
        knn_classifier = selectModelParameters(X_train, y_train, clf, parameters, scorer, 
                                               list_participants_65=list_participants_65, LOPO=True)
        
        # X datasets still have the Participant_No, so remove the first column
        X_train = X_train[:,1:]
        X_test = X_test[:,1:]
        
    else:
        knn_classifier = selectModelParameters(X_train, y_train, clf, parameters, scorer)

    # fitting the model
    knn_classifier.fit(X_train, y_train)

    print(knn_classifier)

    # predict the response
    y_pred = knn_classifier.predict(X_test)

    # evaluate accuracy
    knn_acc = getClfMetrics(y_test, y_pred)

    return knn_acc, knn_classifier


In [11]:
def buildTrainSVM(dataframe):
    """
    Breakdown the dataframe into X and y arrays. Later split them in train and test set. Train the model with CV
    and report the accuracy
    """

    # create design matrix X and target vector y
    X = np.array(dataframe.iloc[:, 0:dataframe.shape[1] - 1]) # minus 1 for the comfort label
    y = np.array(dataframe['Discrete Thermal Comfort_TA'])

    print("Features: {}".format(dataframe.columns.values[:-1]))  # minus 1 for the comfort label

    scaler = StandardScaler()
    scaled_X = scaler.fit_transform(X) # TODO: does this take care of boolean variables?

    # split into train and test
    test_size_percentage = 0.2

    # X_train = train + cv set (train_vectors)
    # X_test = test set (test_vectors)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size = test_size_percentage, random_state = 100, stratify = y)

    parameters = [{'C' : [1, 10, 100, 1000],
                   'kernel' : ['linear'], 
                   'class_weight' : ['balanced']},
                  {'C' : [1, 10, 100, 1000], 
                   'kernel' : ['rbf'], 
                   'gamma' : [0.1, 0.01, 0.001, 0.0001], 
                   'class_weight' : ['balanced']}]
    clf = SVC(C = 1, kernel = 'linear', class_weight = None, random_state = 100)
    scorer = 'f1_micro'
    svm_classifier = selectModelParameters(X_train, y_train, clf, parameters, scorer)

    # fitting the model
    svm_classifier.fit(X_train, y_train)
    print(svm_classifier)

    # predict the response
    y_pred = svm_classifier.predict(X_test)

    # evaluate accuracy
    svm_acc = getClfMetrics(y_test, y_pred)
    
    return svm_acc, svm_classifier


In [12]:
def buildTrainNB(dataframe):
    """
    Breakdown the dataframe into X and y arrays. Later split them in train and test set. Train the model with CV
    and report the accuracy
    """
    
    DEFAULT_K = 10
    
    # create design matrix X and target vector y
    X = np.array(dataframe.iloc[:, 0:dataframe.shape[1] - 1]) # minus 1 for the comfort label
    y = np.array(dataframe['Discrete Thermal Comfort_TA'])

    print("Features: {}".format(dataframe.columns.values[:-1]))  # minus 1 for the comfort label

    scaler = StandardScaler()
    scaled_X = scaler.fit_transform(X) # TODO: does this take care of boolean variables?

    # split into train and test
    test_size_percentage = 0.2

    # X_train = train + cv set
    # X_test = test set
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size = test_size_percentage, random_state = 100, stratify = y)

    # instantiate learning model
    nb_classifier = GaussianNB() # TODO get priors?

    # k-fold cross validation
    scores = cross_val_score(nb_classifier, X_train, y_train, cv = DEFAULT_K, scoring = 'accuracy') # accuracy here is f1 micro
    print("Expected accuracy (f1 micro) based on Cross-Validation: ", scores.mean())

    # fitting the model
    nb_classifier.fit(X_train, y_train)
    print(nb_classifier)

    # predict the response
    y_pred = nb_classifier.predict(X_test)

    # Metrics
    nb_acc = getClfMetrics(y_test, y_pred)
    
    return nb_acc , nb_classifier


In [13]:
def buildMLP(dataframe, binary=False):
    """
    Breakdown the dataframe into X and y arrays. Later split them in train and test set. Train the model with CV
    and report the accuracy
    """
    
    DEFAULT_K = 10
    
    # create design matrix X and target vector y
    X = np.array(dataframe.iloc[:, 0:dataframe.shape[1] - 1]) # minus 1 for the comfort label
    y_original = np.array(dataframe['Discrete Thermal Comfort_TA'])

    # re-map labels from {-2, -1, 0, 1, 2} to {0, 1, 2, 3, 4}; works better with _tanh_ activation
    y_norm = y_original + 2
    
    # binary case
    if binary:
        y_norm = dataframe['Discrete Thermal Comfort_TA'].map(lambda x: 1 if x != 0 else 0)
    
    # generate one-hot vector representation; e.g., 0 = [1 0 0 0 0], 1 = [0 1 0 0 0], etc.
    y = np.zeros((y_norm.size, y_norm.max()+1)) 
    y[np.arange(y_norm.size),y_norm] = 1 

    scaler = StandardScaler()
    scaled_X = scaler.fit_transform(X) # TODO: does this take care of boolean variables?

    # split into train and test
    test_size_percentage = 0.2 # standard 80/20 split

    # X_train = train + cv set
    # X_test = test set
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size = test_size_percentage, random_state = 100, stratify = y)

    # MLP is sensitive to feature scaling. 
    mlp = MLPClassifier(hidden_layer_sizes = (100,100,25,5), 
                    activation = 'tanh', 
                    solver = 'lbfgs',
                    learning_rate = 'adaptive',
                    alpha = 1e-4, 
                    learning_rate_init = 1e-3, # only used with solver adam or sgd
                    max_iter = 1000,
                    tol = 1e-8,
                    batch_size = 5,
                    random_state = 100,
                    verbose = True,
                    warm_start = False,
                    nesterovs_momentum = True,
                    shuffle = True)

    # model learned
    print(mlp)

    # k-fold cross validation
    scores = cross_val_score(mlp, X_train, y_train, cv = DEFAULT_K, scoring = 'accuracy') # accuracy here is f1 micro
    print("Expected accuracy (f1 micro) based on Cross-Validation: ", scores.mean(), "\n")

    # fitting the model
    mlp.fit(X_train, y_train)

    # predict the response
    y_pred = mlp.predict(X_test)
    
    # reverse one hot encoding
    y_pred_normal = []
    y_test_normal = []
    
    if binary:
        for i in range(0,len(y_test)):
            if y_pred[i,0] == 1:
                y_pred_normal.append(0)
            elif y_pred[i,1] == 1:
                y_pred_normal.append(1)
            else:
                y_pred_normal.append(0)
                
            if y_test[i,0] == 1:
                y_test_normal.append(0)
            elif y_test[i,1] ==1:
                y_test_normal.append(1)
            else:
                y_test_normal.append(0)
        
    else: 
        for i in range(0,len(y_test)):
            if y_pred[i,0] == 1:
                y_pred_normal.append(-2)
            elif y_pred[i,1] == 1:
                y_pred_normal.append(-1)
            elif y_pred[i,2] == 1:
                y_pred_normal.append(0)
            elif y_pred[i,3] == 1:
                y_pred_normal.append(1)
            elif y_pred[i,4] == 1:
                y_pred_normal.append(2)
            else:
                y_pred_normal.append(0)

            if y_test[i,0] == 1:
                y_test_normal.append(-2)
            elif y_test[i,1] ==1:
                y_test_normal.append(-1)
            elif y_test[i,2] == 1:
                y_test_normal.append(0)
            elif y_test[i,3] == 1:
                y_test_normal.append(1)
            elif y_test[i,4] == 1:
                y_test_normal.append(2)
            else:
                y_pred_normal.append(0)
    

    y_pred_normal = np.array(y_pred_normal)
    y_test_normal = np.array(y_test_normal)
    
    # Metrics
    mlp_acc = getClfMetrics(y_test_normal, y_pred_normal)
    return mlp_acc, mlp


In [15]:
def saveModel(fileName, model):
    pickle.dump(model, open(fileName, 'wb'))
    